In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import eli5
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from matplotlib.ticker import MaxNLocator
from eli5.sklearn import PermutationImportance
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

In [2]:
#Functions
def weighStats(row, dayWeight, locWeight):
    weight = 1 
    if row['DayNum'] > (132 * 0.75):
        weight += dayWeight
    if (row['WLoc'] == "A") or (row['WLoc'] == "N"):
        weight += locWeight
    # print(row)
    
    # Weighting winner stats
    row['Wsos'] = row['Wsos'] * weight
    row['Wposs'] = row['Wposs'] * weight
    row['Wshoot_eff'] = row['Wshoot_eff'] * weight 
    row['Wscore_op'] = row['Wscore_op'] * weight 
    row['Woff_rtg'] = row['Woff_rtg'] * weight 
    row['Wdef_rtg'] = row['Wdef_rtg'] * weight 
    row['Wts_pct'] = row['Wts_pct'] * weight 
    row['Wefg_pct'] = row['Wefg_pct'] * weight 
    row['Worb_pct'] = row['Worb_pct'] * weight 
    row['Wdrb_pct'] = row['Wdrb_pct'] * weight 
    row['Wreb_pct'] = row['Wreb_pct'] * weight 
    row['Wto_poss'] = row['Wto_poss'] * weight 
    row['Wft_rate'] = row['Wft_rate'] * weight 
    row['Wast_rtio'] = row['Wast_rtio'] * weight
    row['Wblk_pct'] = row['Wblk_pct'] * weight
    row['Wstl_pct'] = row['Wstl_pct'] * weight
    
    # Weighting Losses
#     row['Lposs'] = row['Lposs'] * weight
#     row['Lshoot_eff'] = row['Lshoot_eff'] * weight
#     row['Lscore_op'] = row['Lscore_op'] * weight
#     row['Loff_rtg'] = row['Loff_rtg'] * weight
#     row['Ldef_rtg'] = row['Ldef_rtg'] * weight
#     row['Lts_pct'] = row['Lts_pct'] * weight
#     row['Lefg_pct'] = row['Lefg_pct'] * weight
#     row['Lorb_pct'] = row['Lorb_pct'] * weight
#     row['Ldrb_pct'] = row['Ldrb_pct'] * weight
#     row['Lreb_pct'] = row['Lreb_pct'] * weight
#     row['Lto_poss'] = row['Lto_poss'] * weight
#     row['Lft_rate'] = row['Lft_rate'] * weight
#     row['Last_rtio'] = row['Last_rtio'] * weight
#     row['Lblk_pct'] = row['Lblk_pct'] * weight
#     row['Lstl_pct'] = row['Lstl_pct'] * weight

    return row


def calcAverages(df):
    df_avg = pd.DataFrame()

    df_avg['n_wins'] = df['WTeamID'].groupby([df.Season, df.WTeamID, df.WTeamName, df.WConfName]).count()
    df_avg['n_loss'] = df['LTeamID'].groupby([df.Season, df.LTeamID, df.LTeamName, df.LConfName]).count()

    df_avg['n_loss'].fillna(0, inplace=True)

    # Calculate win percentages:
    df_avg['win_pct'] = df_avg['n_wins'] / (df_avg['n_wins'] + df_avg['n_loss'])
    # Calculate averages for games won:
    df_avg['Wshoot_eff'] = df['Wshoot_eff'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wscore_op'] = df['Wscore_op'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Woff_rtg'] = df['Woff_rtg'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wdef_rtg'] = df['Wdef_rtg'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wsos'] = df['Wsos'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wts_pct'] = df['Wts_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wefg_pct'] = df['Wefg_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Worb_pct'] = df['Worb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wdrb_pct'] = df['Wdrb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wreb_pct'] = df['Wreb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wto_poss'] = df['Wto_poss'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wft_rate'] = df['Wft_rate'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wie'] = df['Wie'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wast_rtio'] = df['Wast_rtio'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wblk_pct'] = df['Wblk_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Wstl_pct'] = df['Wstl_pct'].groupby([df['Season'], df['WTeamID']]).mean()

    # Calculate averages for games lost:
    df_avg['Lshoot_eff'] = df['Lshoot_eff'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lscore_op'] = df['Lscore_op'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Loff_rtg'] = df['Loff_rtg'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Ldef_rtg'] = df['Ldef_rtg'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lsos'] = df['Lsos'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lts_pct'] = df['Lts_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lefg_pct'] = df['Lefg_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lorb_pct'] = df['Lorb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Ldrb_pct'] = df['Ldrb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lreb_pct'] = df['Lreb_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lto_poss'] = df['Lto_poss'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lft_rate'] = df['Lft_rate'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lie'] = df['Lie'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Last_rtio'] = df['Last_rtio'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lblk_pct'] = df['Lblk_pct'].groupby([df['Season'], df['WTeamID']]).mean()
    df_avg['Lstl_pct'] = df['Lstl_pct'].groupby([df['Season'], df['WTeamID']]).mean()

    return df_avg


def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def predictMatches(clf, df_predict, df_features, year):
    diff = []
    data = []

    for i, row in df_predict.iterrows():
        year, team1, team2 = get_year_t1_t2(row.ID)
#         print("Year: {}; Team1: {}; Team2: {}".format(year, team1, team2))
#         Save 2018 stats/features for the first ID:
        team1 = df_features[(df_features['Season'] == year) & (df_features['TeamID'] == team1)].values[0]

        # Save 2018 stats/features for the first ID:
        team2 = df_features[(df_features['Season'] == year) & (df_features['TeamID'] == team2)].values[0]   

        diff = team1 - team2

        data.append(diff)

    n_poss_games = len(df_predict)
    columns = df_features.columns.get_values()
#     print(columns)
    final_predictions = pd.DataFrame(np.array(data).reshape(n_poss_games, np.array(data).shape[1]), columns=(columns))
    final_predictions.drop(['Season', 'TeamID'], inplace=True, axis=1)
    predictions = clf.predict_proba(final_predictions)[:, 1]
#     predLogLoss = log_loss(y_test, search.predict_proba(final_predictions))
    clipped_predictions = np.clip(predictions, 0.05, 0.95)
    df_predict.Pred = clipped_predictions
    
    return df_predict


# Source: https://www.kaggle.com/goodspellr/how-to-score-your-own-predictions-and-more
def kaggle_clip_log(x):
    '''
    Calculates the natural logarithm, but with the argument clipped within [1e-15, 1 - 1e-15]
    '''
    return np.log(np.clip(x,1.0e-15, 1.0 - 1.0e-15))

def kaggle_log_loss(pred, result):
    '''
    Calculates the kaggle log loss for prediction pred given result result
    '''
    return -(result*kaggle_clip_log(pred) + (1-result)*kaggle_clip_log(1.0 - pred))
    
def score_submission(df_sub, df_results, on_season=None, return_df_analysis=True):
    '''
    Scores a submission against relevant tournament results
    
    Parameters
    ==========
    df_sub: Pandas dataframe containing predictions to be scored (must contain a column called 'ID' and 
            a column called 'Pred')
            
    df_results: Pandas dataframe containing results to be compared against (must contain a column 
            called 'ID' and a column called 'Result')
            
    on_season: array-like or None.  If array, should contain the seasons for which a score should
            be calculated.  If None, will use all seasons present in df_results
            
    return_df_analysis: Bool.  If True, will return the dataframe used for calculations.  This is useful
            for future analysis
            
    Returns
    =======
    df_score: pandas dataframe containing the average score over predictions that were scorable per season
           as well as the number of obvious errors encountered
    df_analysis:  pandas dataframe containing information about all results used in scoring
                  Only provided if return_df_analysis=True
    '''
    
    df_analysis = df_results.copy()
    
    # this will overwrite if there's already a season column but it should be the same
    df_analysis['Season'] = [int(x.split('_')[0]) for x in df_results['ID']]
    
    if not on_season is None:
        df_analysis = df_analysis[np.in1d(df_analysis['Season'], on_season)]
        
    # left merge with the submission.  This will keep all games for which there
    # are results regardless of whether there is a prediction
    df_analysis = df_analysis.merge(df_sub, how='left', on='ID')
    
    # check to see if there are obvious errors in the predictions:
    # Obvious errors include predictions that are less than 0, greater than 1, or nan
    # You can add more if you like
    df_analysis['ObviousError'] = 1*((df_analysis['Pred'] < 0.0) \
                                  | (df_analysis['Pred'] > 1.0) \
                                  | (df_analysis['Pred'].isnull()))
    
    df_analysis['LogLoss'] = kaggle_log_loss(df_analysis['Pred'], df_analysis['Result'])
    
    df_score = df_analysis.groupby('Season').agg({'LogLoss' : 'mean', 'ObviousError': 'sum'})
    
    if return_df_analysis:
        return df_score, df_analysis
    else:
        return df_score
    
def convertTourneyRound(dayNum):
    """
    Consolidate tournament rounds into meaningful info.
    """
    if (dayNum == 136) | (dayNum == 137):
        return 64
    elif (dayNum == 138) | (dayNum == 139):
        return 32
    elif (dayNum == 143) | (dayNum == 144):
        return 16
    elif (dayNum == 145) | (dayNum == 146):
        return 8
    elif dayNum == 152:
        return 4
    elif dayNum == 154:
        return 2
    else:
        return 68
    
def processSeeds(df):
    df['seed'] = df_tourney_seeds['Seed'].apply(lambda x : int(x[1:3]))
    df.drop(columns={'Seed'}, inplace=True)
    return df

###### Column names
* WFGM - field goals made (by the winning team)
* WFGA - field goals attempted (by the winning team)
* WFGM3 - three pointers made (by the winning team)
* WFGA3 - three pointers attempted (by the winning team)
* WFTM - free throws made (by the winning team)
* WFTA - free throws attempted (by the winning team)
* WOR - offensive rebounds (pulled by the winning team)
* WDR - defensive rebounds (pulled by the winning team)
* WAst - assists (by the winning team)
* WTO - turnovers committed (by the winning team)
* WStl - steals (accomplished by the winning team)
* WBlk - blocks (accomplished by the winning team)
* WPF - personal fouls committed (by the winning team)

In [3]:
stage2DataPath = 'data\\Stage2DataFiles\\'
dataPath = 'data\\'
# Read in detail tournment results from 2018 and previous
df_tourneyDetailedResults = pd.read_csv(stage2DataPath + "NCAATourneyDetailedResults.csv")
# Read in the team names and teamID
df_regularSeason_teams = pd.read_csv( stage2DataPath + "Teams.csv")
# Read in the regular season results
df_regularSeason = pd.read_csv(stage2DataPath + "RegularSeasonDetailedResults.csv")
# Read in the Tournment seed data
df_tourney_seeds = pd.read_csv(stage2DataPath + "NCAATourneySeeds.csv")
# Read in Conference information
df_team_conf = pd.read_csv(stage2DataPath + 'TeamConferences.csv')
df_conf = pd.read_csv(dataPath + "Conferences.csv")
df_tourneyCompact = pd.read_csv(stage2DataPath + "NCAATourneyCompactResults.csv")

In [4]:
df_tourney_seeds = processSeeds(df_tourney_seeds)

In [5]:
df_regularSeason.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [6]:
# Dropping DayNum, WLoc, NumOT
# df_regularSeason.drop(columns={'DayNum', 'WLoc','NumOT'}, inplace=True)
# df_regularSeason.drop(columns={'NumOT'}, inplace=True)


In [7]:
def buildRegularSeason(df_regularSeason, df_conf, df_team_conf):
    #     Merge team conference with conference
    df_regularSeason_conf_name = df_team_conf.merge(df_conf, on=['ConfAbbrev'])

    # Matching up winning and losing teamIDs
    win_teams = df_regularSeason_teams.rename(columns={'TeamID':'WTeamID'})[['WTeamID', 'TeamName']]
    win_confs = df_regularSeason_conf_name.rename(columns={'TeamID':'WTeamID'})[['Season', 'WTeamID', 'Description']]
    lose_teams = df_regularSeason_teams.rename(columns={'TeamID':'LTeamID'})[['LTeamID', 'TeamName']]
    lose_confs = df_regularSeason_conf_name.rename(columns={'TeamID':'LTeamID'})[['Season', 'LTeamID', 'Description']]

    # Merge conference and team name of winning team and losing team
    df_regularSeason = df_regularSeason.merge(win_teams, on='WTeamID').rename(columns={'TeamName': 'WTeamName'}) \
    .merge(win_confs, on=['Season', 'WTeamID']).rename(columns={'Description': 'WConfName'}) \
    .merge(lose_teams, on='LTeamID').rename(columns={'TeamName': 'LTeamName'}) \
    .merge(lose_confs, on=['Season', 'LTeamID']).rename(columns={'Description': 'LConfName'})

    # Build two point Field goal information.
    # Providing data includes 3 point shots in the field goal data, so FG - FG3 will get the two point field goal data.
    df_regularSeason['WFGM2'] = df_regularSeason.WFGM - df_regularSeason.WFGM3
    df_regularSeason['WFGA2'] = df_regularSeason.WFGA - df_regularSeason.WFGA3
    df_regularSeason['LFGM2'] = df_regularSeason.LFGM - df_regularSeason.LFGM3
    df_regularSeason['LFGA2'] = df_regularSeason.LFGA - df_regularSeason.LFGA3

    # Winner stats related to offensive efficiency:
    df_regularSeason['Wposs'] = df_regularSeason.apply(lambda row: row.WFGA + 0.475 * row.WFTA + row.WTO - row.WOR, axis=1)
    df_regularSeason['Wshoot_eff'] = df_regularSeason.apply(lambda row: row.WScore / (row.WFGA + 0.475 * row.WFTA), axis=1)
    df_regularSeason['Wscore_op'] = df_regularSeason.apply(lambda row: (row.WFGA + 0.475 * row.WFTA) / row.Wposs, axis=1)
    df_regularSeason['Woff_rtg'] = df_regularSeason.apply(lambda row: row.WScore / row.Wposs*100, axis=1)

    # Loser stats related to offensive efficiency:
    df_regularSeason['Lposs'] = df_regularSeason.apply(lambda row: row.LFGA + 0.475 * row.LFTA + row.LTO - row.LOR, axis=1)
    df_regularSeason['Lshoot_eff'] = df_regularSeason.apply(lambda row: row.LScore / (row.LFGA + 0.475 * row.LFTA), axis=1)
    df_regularSeason['Lscore_op'] = df_regularSeason.apply(lambda row: (row.LFGA + 0.475 * row.LFTA) / row.Lposs, axis=1)
    df_regularSeason['Loff_rtg'] = df_regularSeason.apply(lambda row: row.LScore/row.Lposs*100, axis=1)

    # Defensive and net efficiency:
    df_regularSeason['Wdef_rtg'] = df_regularSeason.apply(lambda row: row.Loff_rtg, axis=1)
    df_regularSeason['Wsos'] = df_regularSeason.apply(lambda row: row.Woff_rtg - row.Loff_rtg, axis=1)
    df_regularSeason['Ldef_rtg'] = df_regularSeason.apply(lambda row: row.Woff_rtg, axis=1)
    df_regularSeason['Lsos'] = df_regularSeason.apply(lambda row: row.Loff_rtg - row.Woff_rtg, axis=1)

    # Impact Estimate - 
    # First calculate the teams' overall statistical contribution (the numerator):
    Wie = df_regularSeason.apply(lambda row: row.WScore + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + (0.5 * row.WOR) + row.WAst + row.WStl + (0.5 * row.WBlk) - row.WPF - row.WTO, axis=1)
    Lie = df_regularSeason.apply(lambda row: row.LScore + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + (0.5 * row.LOR) + row.LAst + row.LStl + (0.5 * row.LBlk) - row.LPF - row.LTO, axis=1)

    # Then divide by the total game statistics (the denominator):
    df_regularSeason['Wie'] = Wie / (Wie + Lie) * 100
    df_regularSeason['Lie'] = Lie / (Lie + Wie) * 100

    # Other winner stats:
    df_regularSeason['Wts_pct'] = df_regularSeason.apply(lambda row: row.WScore / (2 * (row.WFGA + 0.475 * row.WFTA)) * 100, axis=1)
    df_regularSeason['Wefg_pct'] = df_regularSeason.apply(lambda row: (row.WFGM2 + 1.5 * row.WFGM3) / row.WFGA, axis=1)
    df_regularSeason['Worb_pct'] = df_regularSeason.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
    df_regularSeason['Wdrb_pct'] = df_regularSeason.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
    df_regularSeason['Wreb_pct'] = df_regularSeason.apply(lambda row: (row.Worb_pct + row.Wdrb_pct) / 2, axis=1)
    df_regularSeason['Wto_poss'] = df_regularSeason.apply(lambda row: row.WTO / row.Wposs, axis=1)
    df_regularSeason['Wft_rate'] = df_regularSeason.apply(lambda row: row.WFTM / row.WFGA, axis=1)
    df_regularSeason['Wast_rtio'] = df_regularSeason.apply(lambda row: row.WAst / (row.WFGA + 0.475*row.WFTA + row.WTO + row.WAst) * 100, axis=1)
    df_regularSeason['Wblk_pct'] = df_regularSeason.apply(lambda row: row.WBlk / row.LFGA2 * 100, axis=1)
    df_regularSeason['Wstl_pct'] = df_regularSeason.apply(lambda row: row.WStl / row.Lposs * 100, axis=1)

    # Other loser stats:
    df_regularSeason['Lts_pct'] = df_regularSeason.apply(lambda row: row.LScore / (2 * (row.LFGA + 0.475 * row.LFTA)) * 100, axis=1)
    df_regularSeason['Lefg_pct'] = df_regularSeason.apply(lambda row: (row.LFGM2 + 1.5 * row.LFGM3) / row.LFGA, axis=1)
    df_regularSeason['Lorb_pct'] = df_regularSeason.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
    df_regularSeason['Ldrb_pct'] = df_regularSeason.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)
    df_regularSeason['Lreb_pct'] = df_regularSeason.apply(lambda row: (row.Lorb_pct + row.Ldrb_pct) / 2, axis=1)
    df_regularSeason['Lto_poss'] = df_regularSeason.apply(lambda row: row.LTO / row.Lposs, axis=1)
    df_regularSeason['Lft_rate'] = df_regularSeason.apply(lambda row: row.LFTM / row.LFGA, axis=1)
    df_regularSeason['Last_rtio'] = df_regularSeason.apply(lambda row: row.LAst / (row.LFGA + 0.475*row.LFTA + row.LTO + row.LAst) * 100, axis=1)
    df_regularSeason['Lblk_pct'] = df_regularSeason.apply(lambda row: row.LBlk / row.WFGA2 * 100, axis=1)
    df_regularSeason['Lstl_pct'] = df_regularSeason.apply(lambda row: row.LStl / row.Wposs * 100, axis=1)
 
    return df_regularSeason, win_teams, win_confs, lose_teams, lose_confs


In [8]:
# def buildRegSeasonStats(df_regularSeason):
#     # Other winner stats:
#     df_regularSeason['Wts_pct'] = df_regularSeason.apply(lambda row: row.WScore / (2 * (row.WFGA + 0.475 * row.WFTA)) * 100, axis=1)
#     df_regularSeason['Wefg_pct'] = df_regularSeason.apply(lambda row: (row.WFGM2 + 1.5 * row.WFGM3) / row.WFGA, axis=1)
#     df_regularSeason['Worb_pct'] = df_regularSeason.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
#     df_regularSeason['Wdrb_pct'] = df_regularSeason.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
#     df_regularSeason['Wreb_pct'] = df_regularSeason.apply(lambda row: (row.Worb_pct + row.Wdrb_pct) / 2, axis=1)
#     df_regularSeason['Wto_poss'] = df_regularSeason.apply(lambda row: row.WTO / row.Wposs, axis=1)
#     df_regularSeason['Wft_rate'] = df_regularSeason.apply(lambda row: row.WFTM / row.WFGA, axis=1)
#     df_regularSeason['Wast_rtio'] = df_regularSeason.apply(lambda row: row.WAst / (row.WFGA + 0.475*row.WFTA + row.WTO + row.WAst) * 100, axis=1)
#     df_regularSeason['Wblk_pct'] = df_regularSeason.apply(lambda row: row.WBlk / row.LFGA2 * 100, axis=1)
#     df_regularSeason['Wstl_pct'] = df_regularSeason.apply(lambda row: row.WStl / row.Lposs * 100, axis=1)

#     # Other loser stats:
#     df_regularSeason['Lts_pct'] = df_regularSeason.apply(lambda row: row.LScore / (2 * (row.LFGA + 0.475 * row.LFTA)) * 100, axis=1)
#     df_regularSeason['Lefg_pct'] = df_regularSeason.apply(lambda row: (row.LFGM2 + 1.5 * row.LFGM3) / row.LFGA, axis=1)
#     df_regularSeason['Lorb_pct'] = df_regularSeason.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#     df_regularSeason['Ldrb_pct'] = df_regularSeason.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)
#     df_regularSeason['Lreb_pct'] = df_regularSeason.apply(lambda row: (row.Lorb_pct + row.Ldrb_pct) / 2, axis=1)
#     df_regularSeason['Lto_poss'] = df_regularSeason.apply(lambda row: row.LTO / row.Lposs, axis=1)
#     df_regularSeason['Lft_rate'] = df_regularSeason.apply(lambda row: row.LFTM / row.LFGA, axis=1)
#     df_regularSeason['Last_rtio'] = df_regularSeason.apply(lambda row: row.LAst / (row.LFGA + 0.475*row.LFTA + row.LTO + row.LAst) * 100, axis=1)
#     df_regularSeason['Lblk_pct'] = df_regularSeason.apply(lambda row: row.LBlk / row.WFGA2 * 100, axis=1)
#     df_regularSeason['Lstl_pct'] = df_regularSeason.apply(lambda row: row.LStl / row.Wposs * 100, axis=1)
#     df_regularSeason.reset_index(inplace = True)
#     df_regularSeason = df_regularSeason.rename(columns={'Wshoot_eff':'shoot_eff', 'Wscore_op':'score_op', 
#                                     'Woff_rtg':'off_rtg', 'Wdef_rtg':'def_rtg', 'Wsos':'sos', 
#                                     'Wie':'ie', 'Wts_pct':'ts_pct', 'Wefg_pct':'efg_pct', 'Worb_pct':'orb_pct', 
#                                     'Wdrb_pct':'drb_pct', 'Wreb_pct':'reb_pct', 'Wto_poss':'to_poss', 
#                                     'Wft_rate':'ft_rate', 'Wast_rtio':'ast_rtio', 
#                                     'Wblk_pct':'blk_pct', 'Wstl_pct':'stl_pct'})
#     df_regularSeason = df_regularSeason.rename(columns={'WTeamID': 'TeamID', 'WTeamName': 'TeamName', 'WConfName': 'ConfName'})
#     return df_regularSeason

In [9]:
df_regularSeason, win_teams, win_confs, lose_teams, lose_confs = buildRegularSeason(df_regularSeason, df_conf, df_team_conf)

In [10]:
df_regularSeason_weighted = df_regularSeason.apply(weighStats, axis=1, args=(0.52, 0.15))

df_regularSeason_weighted.reset_index(inplace = True)
df_regularSeason_weighted.rename(columns={'Wshoot_eff':'shoot_eff', 'Wscore_op':'score_op', 
                                'Woff_rtg':'off_rtg', 'Wdef_rtg':'def_rtg', 'Wsos':'sos', 
                                'Wie':'ie', 'Wts_pct':'ts_pct', 'Wefg_pct':'efg_pct', 'Worb_pct':'orb_pct', 
                                'Wdrb_pct':'drb_pct', 'Wreb_pct':'reb_pct', 'Wto_poss':'to_poss', 
                                'Wft_rate':'ft_rate', 'Wast_rtio':'ast_rtio', 
                                'Wblk_pct':'blk_pct', 'Wstl_pct':'stl_pct'}, inplace=True)
df_regularSeason_weighted.rename(columns={'WTeamID': 'TeamID', 'WTeamName': 'TeamName', 'WConfName': 'ConfName'}, inplace=True)

    
    

In [11]:
# df_regularSeason_avgs = buildRegSeasonStats(df_regularSeason_weighted)
df_regularSeason_weighted.columns

Index(['index', 'Season', 'DayNum', 'TeamID', 'WScore', 'LTeamID', 'LScore',
       'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
       'LBlk', 'LPF', 'TeamName', 'ConfName', 'LTeamName', 'LConfName',
       'WFGM2', 'WFGA2', 'LFGM2', 'LFGA2', 'Wposs', 'shoot_eff', 'score_op',
       'off_rtg', 'Lposs', 'Lshoot_eff', 'Lscore_op', 'Loff_rtg', 'def_rtg',
       'sos', 'Ldef_rtg', 'Lsos', 'ie', 'Lie', 'ts_pct', 'efg_pct', 'orb_pct',
       'drb_pct', 'reb_pct', 'to_poss', 'ft_rate', 'ast_rtio', 'blk_pct',
       'stl_pct', 'Lts_pct', 'Lefg_pct', 'Lorb_pct', 'Ldrb_pct', 'Lreb_pct',
       'Lto_poss', 'Lft_rate', 'Last_rtio', 'Lblk_pct', 'Lstl_pct'],
      dtype='object')

In [12]:
# fig, ax = plt.subplots(figsize=(20,15))
# y_bar = df_regularSeason["WConfName"].value_counts().tolist()
# x_bar = df_regularSeason["WConfName"].value_counts().keys()
# ax.barh(x_bar, y_bar, align='center',
#         color='green', ecolor='black')
# # ax.set_yticks(y_bar)
# # ax.set_yticklabels(x_bar)
# ax.invert_yaxis()  # labels read top-to-bottom
# ax.set_xlabel('Wins')
# ax.set_ylabel('Conference')
# ax.set_title('Number of NCAA Division I wins per conference')

# plt.show()

In [13]:
# # temp = df_regularSeason_regularSeason.Season.value_counts().tolist()
# y_bar = df_regularSeason["Season"].value_counts().tolist()
# x_bar = df_regularSeason["Season"].value_counts().keys()

# fig, ax = plt.subplots(figsize=(15,5))
# ax.barh(x_bar, y_bar, align='center',
#         color='green', ecolor='black')
# # ax.set_yticks(y_bar)
# # ax.set_yticklabels(x_bar)
# # ax.invert_yaxis()  # labels read top-to-bottom
# ax.set_xlabel('Games')
# ax.set_ylabel('Season')
# ax.set_title('Number of Games per Season')

# plt.show()


### Feature Engineering

In [14]:
# Shooting Efficiency = Score / ( FGA + 0.475 * FTA )
# Scoring Opportunity = (FGA + 0.475 FTA)/(poss)
# Defensive Rating = opponent's offensive rating
# Offensive Rating = (score / poss) * 100
# Net Efficiency = offensive rating - opponent offensive rating

In [15]:
# Shooting features
# True Shooting percentage = ((score)/(2 * (FGA + 0.475 * FTA))) * 100
# Effective Field Goal Percentage = (FGM2 + 1.5 * FGM3) / (FGA)

In [16]:
# Rebounding features
# Offensive Rebound Percentage(orb_pct) = (OR) / (OR + opponent DR)
# Defensive Rebound Percentage(drb_pct) = (DR) / (DR + opponent OR)
# Rebound Percentage(reb_pct) = (orb_pct + drb_pct)/2

In [17]:
# Basketball on Paper author Dean Oliver outlines four factors that determine success in basketball:
# Effective Field Goal percentage
# Turnovers per possession(to_poss) = Turnovers / poss
# Offensive Rebound Percentage
# Free throw rate(ft_rate) = FTM / FGA

In [18]:
# https://stats.nba.com/help/faq/ - NBA Advanced Stats page
# Team Impact Estimate
# IE_numerator = Score + FGM + FTM - FGA - FTA + DR + 0.5 * OR + Ast + Stl + 0.5 * Blk - PF - TO
# IE = IE_numerator / (IE_Numerator + opp_IE_numerator)

# Assist Ratio(ast_rat) = (Ast / (FGA + 0.475 * FTA + TO + Ast)) * 100
# Block Percentage(blk_pct) = (Blk / opp_FGA2) * 100
# Steal Percentage(stl_pct) = (Stl / opp_poss) * 100


In [19]:
# df_regularSeason.head()


In [20]:
# df_regularSeason_weighted = df_regularSeason.copy()


In [21]:
df_regularSeason_weighted.head()

,index,Season,DayNum,TeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,...,Lts_pct,Lefg_pct,Lorb_pct,Ldrb_pct,Lreb_pct,Lto_poss,Lft_rate,Last_rtio,Lblk_pct,Lstl_pct
0,0,2003,10,1104,68,1328,62,N,0,27,...,48.857368,0.433962,0.294118,0.611111,0.452614,0.251924,0.301887,8.943544,4.545455,11.912641
1,1,2003,98,1400,67,1328,61,H,0,25,...,51.585624,0.490385,0.272727,0.581395,0.427061,0.218324,0.192308,6.400000,6.000000,7.633588
2,2,2003,124,1400,76,1328,71,A,0,27,...,50.052873,0.483333,0.348837,0.678571,0.513704,0.138621,0.216667,16.679698,15.384615,12.144213
3,3,2003,114,1281,67,1328,52,H,0,24,...,40.609137,0.318182,0.414634,0.687500,0.551067,0.189573,0.309091,9.635652,7.500000,1.765225
4,4,2003,70,1329,48,1328,46,H,0,19,...,37.022133,0.354545,0.348837,0.696970,0.522903,0.112941,0.127273,12.800000,7.317073,7.326007


In [22]:
# df_regularSeason_avgs = calcAverages(df=df_regularSeason_weighted)

In [23]:
# df_regularSeason_weighted[(df_regularSeason_weighted['WTeamID'] == 1104) & (df_regularSeason_weighted['DayNum'] <= 99) & 
#                               (df_regularSeason_weighted['Season'] == 2003)]
# .groupby(df_regularSeason_weighted['Season']).sum()
# 
# temp_df.Lts_pct

In [24]:
# Calculate weighted average using win percentage:
# df_regularSeason_avgs['shoot_eff'] = df_regularSeason_avgs['Wshoot_eff'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lshoot_eff'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['score_op'] = df_regularSeason_avgs['Wscore_op'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lscore_op'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['off_rtg'] = df_regularSeason_avgs['Woff_rtg'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Loff_rtg'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['def_rtg'] = df_regularSeason_avgs['Wdef_rtg'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Ldef_rtg'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['sos'] = df_regularSeason_avgs['Wsos'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lsos'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['ts_pct'] = df_regularSeason_avgs['Wts_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lts_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['efg_pct'] = df_regularSeason_avgs['Wefg_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lefg_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['orb_pct'] = df_regularSeason_avgs['Worb_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lorb_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['drb_pct'] = df_regularSeason_avgs['Wdrb_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Ldrb_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['reb_pct'] = df_regularSeason_avgs['Wreb_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lreb_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['to_poss'] = df_regularSeason_avgs['Wto_poss'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lto_poss'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['ft_rate'] = df_regularSeason_avgs['Wft_rate'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lft_rate'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['ie'] = df_regularSeason_avgs['Wie'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lie'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['ast_rtio'] = df_regularSeason_avgs['Wast_rtio'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Last_rtio'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['blk_pct'] = df_regularSeason_avgs['Wblk_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lblk_pct'] * (1 - df_regularSeason_avgs['win_pct'])
# df_regularSeason_avgs['stl_pct'] = df_regularSeason_avgs['Wstl_pct'] * df_regularSeason_avgs['win_pct'] + df_regularSeason_avgs['Lstl_pct'] * (1 - df_regularSeason_avgs['win_pct'])

# df_regularSeason_avgs.reset_index(inplace = True)
# df_regularSeason_avgs = df_regularSeason_avgs.rename(columns={'WTeamID': 'TeamID', 'WTeamName': 'TeamName', 'WConfName': 'ConfName'})
# df_regularSeason_avgs.head()

In [25]:
def buildTourneyCompact(df_tourneyCompact, df_tourney_seeds, win_teams, win_confs, lose_teams, lose_confs):
    df_tourneyCompact['tourn_round'] = df_tourneyCompact.DayNum.apply(convertTourneyRound)

#     df_tourney_seeds['seed'] = df_tourney_seeds['Seed'].apply(lambda x : int(x[1:3]))
#     df_tourney_seeds.head()

    # Drop Seed column
#     df_tourney_seeds.drop(columns={'Seed'}, inplace=True)

    # df_tourney_seeds.head()

    df_tourneyCompact = df_tourneyCompact.merge(df_tourney_seeds, how='left', 
                                                            left_on=['Season', 'WTeamID'], 
                                                            right_on=['Season', 'TeamID']) \
    .rename(columns={'seed': 'Wseed'}).drop(['TeamID'], axis=1) \
    .merge(df_tourney_seeds, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID']) \
    .rename(columns={'seed': 'Lseed'}).drop(['TeamID'], axis=1) \
    .merge(win_teams, on='WTeamID').rename(columns={'TeamName': 'WTeamName'}) \
    .merge(win_confs, on=['Season', 'WTeamID']).rename(columns={'Description': 'WConfName'}) \
    .merge(lose_teams, on='LTeamID').rename(columns={'TeamName': 'LTeamName'}) \
    .merge(lose_confs, on=['Season', 'LTeamID']).rename(columns={'Description': 'LConfName'})

    # Calculate the point differential:
    df_tourneyCompact['point_diff'] = df_tourneyCompact.WScore - df_tourneyCompact.LScore
    return df_tourneyCompact

In [26]:
# df_tourneyCompact()
# df_tourneyCompact.head()
df_tourney_merge = buildTourneyCompact(df_tourneyCompact, df_tourney_seeds, win_teams, win_confs, lose_teams, lose_confs)

In [27]:
df_tourney_merge.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,tourn_round,Wseed,Lseed,WTeamName,WConfName,LTeamName,LConfName,point_diff
0,1985,136,1116,63,1234,54,N,0,64,9,8,Arkansas,Southwest Conference,Iowa,Big Ten Conference,9
1,1997,138,1246,75,1234,69,N,0,32,1,8,Kentucky,Southeastern Conference,Iowa,Big Ten Conference,6
2,2001,138,1246,92,1234,79,N,0,32,2,7,Kentucky,Southeastern Conference,Iowa,Big Ten Conference,13
3,1987,146,1424,84,1234,81,N,0,8,1,2,UNLV,Pacific Coast Athletic Association,Iowa,Big Ten Conference,3
4,1991,138,1181,85,1234,70,N,0,32,2,7,Duke,Atlantic Coast Conference,Iowa,Big Ten Conference,15


In [28]:
# championships = df_tourney_all[df_tourney_all.tourn_round == 2]

# fig, ax = plt.subplots(figsize=(12, 8))
# sns.countplot(y=championships.WTeamName, order=championships.WTeamName.value_counts().index, color='#3c7f99')
# plt.box(False)

# fig.text(x=-0.05, y=0.95, s='       Championships per Team since 1985       ', fontsize=32, weight='bold', color='white', backgroundcolor='#c5b783')
# plt.title('(When the tournament field expanded to 64 teams.)', fontsize=18)

# plt.tick_params(axis='both', which='both',length=0)
# plt.tick_params(axis='both', which='major', labelsize=16)
# ax.xaxis.grid(which='both', linewidth=0.5, color='#3c7f99')
# plt.xlabel(''), plt.ylabel('');

In [29]:
# fig, ax = plt.subplots(figsize=(12, 8))
# sns.countplot(x=df_tourney_all[df_tourney_all.tourn_round == 64].Wseed, color='#3c7f99')
# plt.box(False)

# fig.text(x=0, y=0.95, s='       First Round Wins by Seed since 1985     ', fontsize=32, weight='bold', color='white', backgroundcolor='#c5b783')
# plt.title('(When the tournament field expanded to 64 teams.)', fontsize=18)

# plt.tick_params(axis='both', which='both',length=0)
# plt.tick_params(axis='both', which='major', labelsize=16)
# ax.yaxis.grid(which='both', linewidth=0.5, color='#3c7f99')
# plt.xlabel(''), plt.ylabel('');

In [30]:
# # Save the correlation matrix:
# matrix = df_regularSeason_avgs[['win_pct', 'shoot_eff', 'score_op', 'off_rtg', 'def_rtg', 'sos', 'ie', 'ts_pct', 'efg_pct', 'orb_pct', 'drb_pct', 'reb_pct', 'to_poss', 'ft_rate', 'ast_rtio', 'blk_pct', 'stl_pct']].corr()

# fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
# # sns.heatmap(df1.iloc[:, 1:6:], annot=True, linewidths=.5, ax=ax)

# # Create mask for the upper triangle:
# mask = np.zeros_like(matrix, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Create a custom diverging colormap:
# cmap = sns.diverging_palette(225, 45, as_cmap=True)

# sns.heatmap(matrix, mask=mask, cmap=cmap, center=0, annot=True, square=True, linewidths=0.25, cbar_kws={'shrink': 0.25}, ax=ax)
# plt.tick_params(axis='both', which='both',length=0)
# plt.tick_params(axis='both', which='major', labelsize=16)

# from scipy.stats.stats import pearsonr
# plt.title('Team Impact Estimate and Win Percentage correlate at an R square of {:0.3f}!'.format(pearsonr(df_regularSeason_avgs.ie, df_regularSeason_avgs.win_pct)[0]), fontsize=24, weight='bold')
# # fig.text(x=0.25, y=0.75, s='In fact, Win Percentage has a moderate or strong positive \nrelationship with most of the efficiency measures.', fontsize=20)

In [31]:
# NOTES 
# https://www.kaggle.com/humburgc/history-eda-machine-learning-march-madness

In [32]:
def buildModelDf(df_regularSeason_avgs, df_tourney_all, df_tourney_seeds):
    df_features = df_regularSeason_avgs[['Season', 'TeamID', 'shoot_eff', 'score_op', 'off_rtg', 'def_rtg', 'sos', 'ie', 'efg_pct', 'to_poss', 'orb_pct', 'ft_rate', 'reb_pct', 'drb_pct', 'ts_pct', 'ast_rtio', 'blk_pct', 'stl_pct']]
    # df_features.head() 
    df_features.dropna(inplace=True)
    
    # Merge seed data into Regular Season data
    df_features = pd.merge(df_tourney_seeds, df_features, how='outer', left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'])
    df_tourney = df_tourney_all[(df_tourney_all.Season >= 2003) & (df_tourney_all.Season < 2019)]
    df_tourney.reset_index(inplace=True, drop=True)

    # Merge tournament winners season features:
    df_winners = pd.merge(left=df_tourney[['Season', 'WTeamID', 'LTeamID']], right=df_features, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
    df_winners.drop(['TeamID'], inplace=True, axis=1) 

    # Merge tournament games with features from losing team:
    df_losers = pd.merge(left=df_tourney[['Season', 'WTeamID', 'LTeamID']], right=df_features, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
    df_losers.drop(['TeamID'], inplace=True, axis=1)

    # Creating a dataframe for the winning teams and assigning 1 to the result column to indiciate the win.
    df_winner_diff = (df_winners.iloc[:, 3:] - df_losers.iloc[:, 3:])
    df_winner_diff['result'] = 1

    # Creating a dataframe for the losing teams and assigning 0 to the result column to indiciate the loss.
    df_loser_diff = (df_losers.iloc[:, 3:] - df_winners.iloc[:, 3:])
    df_loser_diff['result'] = 0

    # Combine winning team data with losing team data
    df_model = pd.concat((df_winner_diff, df_loser_diff), axis=0)
    df_model.dropna(inplace=True)
    return df_model

In [33]:
# KeyError: "['TeamID' 'shoot_eff' 'score_op' 'off_rtg' 'def_rtg' 'sos' 'ie' 'efg_pct'\n 'to_poss' 'orb_pct'
# 'ft_rate' 'reb_pct' 'drb_pct' 'ts_pct' 'ast_rtio'\n 'blk_pct' 'stl_pct'] not in index"
# df_regularSeason_weighted.columns

In [34]:
# Feature Modeling

In [35]:
# df_features.dtypes

In [36]:
# df_tourney_seeds.dtypes

In [37]:

# df_features.head()
# df_features.dtypes

In [38]:
# df_features.head()
# df_features.dropna(inplace=True)

In [39]:
# df_features[df_features.isnull()]

In [40]:
# df_tourney.tail()

In [41]:
df_tourney_all = df_tourneyCompact[df_tourneyCompact.Season < 2018]
# df_tourney_all.head()

In [42]:
df_model = buildModelDf(df_regularSeason_weighted, df_tourney_all, df_tourney_seeds)

In [43]:
# df_features = df_regularSeason_weighted[['Season', 'TeamID', 'shoot_eff', 'score_op', 'off_rtg', 'def_rtg', 'sos', 'ie', 'efg_pct', 'to_poss', 'orb_pct', 'ft_rate', 'reb_pct', 'drb_pct', 'ts_pct', 'ast_rtio', 'blk_pct', 'stl_pct']]
# # df_features.head() 
# # Merge seed data into Regular Season data
# df_features = pd.merge(df_tourney_seeds, df_features, how='outer', left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'])
# df_tourney = df_tourney_all[(df_tourney_all.Season >= 2003) & (df_tourney_all.Season < 2019)]
# df_tourney.reset_index(inplace=True, drop=True)

# # Merge tournament winners season features:
# df_winners = pd.merge(left=df_tourney[['Season', 'WTeamID', 'LTeamID']], right=df_features, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
# df_winners.iloc[:, 3:] 
# df_winners.drop(['TeamID','Seed'], inplace=True, axis=1) 

# # Merge tournament games with features from losing team:
# df_losers = pd.merge(left=df_tourney[['Season', 'WTeamID', 'LTeamID']], right=df_features, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
# df_losers.drop(['TeamID'], inplace=True, axis=1)

# # # Creating a dataframe for the winning teams and assigning 1 to the result column to indiciate the win.
# df_winner_diff = (df_winners.iloc[:, 3:] - df_losers.iloc[:, 3:])
# df_winner_diff['result'] = 1

# # # Creating a dataframe for the losing teams and assigning 0 to the result column to indiciate the loss.
# # df_loser_diff = (df_losers.iloc[:, 3:] - df_winners.iloc[:, 3:])
# # df_loser_diff['result'] = 0

# # # Combine winning team data with losing team data
# # df_model = pd.concat((df_winner_diff, df_loser_diff), axis=0)

In [44]:
# df_regularSeason_weighted.columns

In [45]:
df_model.head()

,seed,shoot_eff,score_op,off_rtg,def_rtg,sos,ie,efg_pct,to_poss,orb_pct,ft_rate,reb_pct,drb_pct,ts_pct,ast_rtio,blk_pct,stl_pct,result
0,0.0,0.036479,0.174482,21.472595,19.413456,2.059139,-0.030686,-0.040205,-0.082291,0.143376,0.036657,0.040587,-0.062201,1.823963,0.895680,5.128205,1.834756,1
1,0.0,0.852515,0.791247,97.862493,109.392657,-11.530163,-20.172554,0.511532,0.028287,0.271863,-0.435231,0.327054,0.382246,42.625760,18.957619,7.211299,0.140612,1
2,0.0,-0.229368,-0.500121,-22.397179,-36.625488,14.228309,12.562858,-0.083215,-0.154587,-0.164510,-0.210496,-0.314255,-0.464000,-11.468383,-2.183798,-10.864475,2.314434,1
3,0.0,0.038865,-0.096485,-6.104706,-11.109810,5.005103,1.800146,0.000000,0.131072,0.092210,-0.120690,0.068327,0.044444,1.943242,8.728278,17.142857,11.301673,1
4,0.0,0.096484,0.146888,24.933643,27.516086,-2.582443,-19.036188,0.018354,-0.116105,-0.010366,0.012236,-0.022913,-0.035461,4.824217,-1.234262,3.935599,4.345091,1


In [46]:
# df_model.columns
# df_model.drop(columns={'seed'}, inplace=True)

In [47]:
# df_model[df_model.isnull()]


In [48]:
# sns.countplot(y)
# plt.xlabel(''), plt.ylabel('')
# plt.xticks([0, 1], ('losses', 'wins'))
# plt.title('The Target Classes are Balanced');

In [49]:
from itertools import permutations 
weights = [x / 100.0 for x in range(0, 105, 5)]
weightPerms = permutations(weights, 2)


In [50]:
for weight in weightPerms:
    x,y = weight
    print("{}, {}".format(x,y))

# Log regression


0.0, 0.05
0.0, 0.1
0.0, 0.15
0.0, 0.2
0.0, 0.25
0.0, 0.3
0.0, 0.35
0.0, 0.4
0.0, 0.45
0.0, 0.5
0.0, 0.55
0.0, 0.6
0.0, 0.65
0.0, 0.7
0.0, 0.75
0.0, 0.8
0.0, 0.85
0.0, 0.9
0.0, 0.95
0.0, 1.0
0.05, 0.0
0.05, 0.1
0.05, 0.15
0.05, 0.2
0.05, 0.25
0.05, 0.3
0.05, 0.35
0.05, 0.4
0.05, 0.45
0.05, 0.5
0.05, 0.55
0.05, 0.6
0.05, 0.65
0.05, 0.7
0.05, 0.75
0.05, 0.8
0.05, 0.85
0.05, 0.9
0.05, 0.95
0.05, 1.0
0.1, 0.0
0.1, 0.05
0.1, 0.15
0.1, 0.2
0.1, 0.25
0.1, 0.3
0.1, 0.35
0.1, 0.4
0.1, 0.45
0.1, 0.5
0.1, 0.55
0.1, 0.6
0.1, 0.65
0.1, 0.7
0.1, 0.75
0.1, 0.8
0.1, 0.85
0.1, 0.9
0.1, 0.95
0.1, 1.0
0.15, 0.0
0.15, 0.05
0.15, 0.1
0.15, 0.2
0.15, 0.25
0.15, 0.3
0.15, 0.35
0.15, 0.4
0.15, 0.45
0.15, 0.5
0.15, 0.55
0.15, 0.6
0.15, 0.65
0.15, 0.7
0.15, 0.75
0.15, 0.8
0.15, 0.85
0.15, 0.9
0.15, 0.95
0.15, 1.0
0.2, 0.0
0.2, 0.05
0.2, 0.1
0.2, 0.15
0.2, 0.25
0.2, 0.3
0.2, 0.35
0.2, 0.4
0.2, 0.45
0.2, 0.5
0.2, 0.55
0.2, 0.6
0.2, 0.65
0.2, 0.7
0.2, 0.75
0.2, 0.8
0.2, 0.85
0.2, 0.9
0.2, 0.95
0.2, 1.0
0.25, 0.0
0.

In [51]:
def buildModel(df_model):
    # Select all columns except the results column 
    X = df_model.iloc[:, :-1]
    # Select the results column
    y = df_model.result

    # Split the dataframe into test and train of 25% test size.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.31, random_state=42)
    
    clf = LogisticRegression(random_state=42)
    grid = {'clf__C': np.logspace(start=-80, stop=40, num=50), 'clf__penalty': ['l1', 'l2'],}
    
    pipe = Pipeline([('clf', clf)])

    # Instantiate grid search using 10-fold cross validation:
    search = GridSearchCV(pipe, grid, cv=10, scoring='neg_log_loss')

    # Learn relationship between predictors (basketball/tourney features) and outcome,
    # and the best parameters for defining such:
    search.fit(X_train, y_train)

    # Predictions on the test set, new data that haven't been introduced to the model:
    predicted = search.predict(X_test)

    # Predictions as probabilities:
    probabilities = search.predict_proba(X_test)[:, 1]
    # Accuracy scores for the training and test sets:
    train_accuracy = search.score(X_train, y_train)
    test_accuracy = search.score(X_test, y_test)

    print('Best Parameters: {}'.format(search.best_params_))
    print('Training Accuracy: {:0.2}'.format(train_accuracy))
    print('Test Accuracy: {:0.2}'.format(test_accuracy))

    # Confusion matrix labels:
    labels = np.array([['true losses','false wins'], ['false losses','true wins']])

    # Model evaluation metrics:
    confusion_mtrx = confusion_matrix(y_test, predicted)
    auc = roc_auc_score(y_test, probabilities)
    fpr, tpr, thresholds = roc_curve(y_test, probabilities)
    logloss = log_loss(y_test, search.predict_proba(X_test))
    print('logloss: {:0.7}'.format(logloss))

    # Plot all metrics in a grid of subplots:
#     fig = plt.figure(figsize=(12, 12))
#     grid = plt.GridSpec(2, 4, wspace=0.75, hspace=0.5)

    # Top-left plot - confusion matrix:
#     plt.subplot(grid[0, :2])
#     sns.heatmap(confusion_mtrx, annot=labels, fmt='')
#     plt.xlabel('Predicted Games')
#     plt.ylabel('Actual Games');

#     # Top-right plot - ROC curve:
#     plt.subplot(grid[0, 2:])
#     plt.plot([0, 1], [0, 1], linestyle='--')
#     plt.plot(fpr, tpr, marker='.')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('AUROC: {:0.3}'.format(auc));

#     # Bottom-left plot - support, or true predictions:
#     plt.subplot(grid[1, :2])
#     sns.countplot(y=predicted, orient='h')
#     plt.yticks([1, 0], ('wins', 'losses'))
#     plt.ylabel(''), plt.xlabel('Number Predicted');

    # Bottom-right plot - classification report:
    # plt.subplot(grid[1, 2:])
    # visualizer = ClassificationReport(search, classes=['losses', 'wins'])
    # visualizer.fit(X_train, y_train)
    # visualizer.score(X_test, y_test)
    # g = visualizer.poof();


In [ ]:
buildModel(df_model)

In [ ]:
pipe = Pipeline([('clf', log_clf)])
    
lr_search = GridSearchCV(pipe, log_param_grid, cv=10)
lr_search.fit(X_train, y_train)
print("Best C: {}; Best penalty: {}".format(lr_search.best_params_['clf__C'], lr_search.best_params_['clf__penalty']))

perm = PermutationImportance(lr_search, random_state=42).fit(X_test, y_test)
eli5.show_weights(perm, feature_names=X_test.columns.tolist())

In [ ]:
df_predict_pre2019 = pd.read_csv(dataPath + 'SampleSubmissionStage1.csv')
df_predictions = pd.read_csv(dataPath + 'SampleSubmissionStage2.csv')
df_predictions.head()

In [ ]:
year = 2019
predictMatches(lr_search, df_predictions, df_features, year).to_csv('best_model_results_{}.csv'.format(year), index=False)
predictMatches(lr_search, df_predict_pre2019, df_features, year).to_csv('best_model_results_pre{}.csv'.format(year), index=False)

In [ ]:
from bracketeer import build_bracket

b = build_bracket(
        outputPath='best_bracket_{}.png'.format(year),
        submissionPath='best_model_results_2019.csv',
        teamsPath= stage2DataPath + 'Teams.csv',
        seedsPath= stage2DataPath + 'NCAATourneySeeds.csv',
        slotsPath= stage2DataPath + 'NCAATourneySlots.csv',
        year=year
)

In [ ]:
## 2019 Predictions
from IPython.display import HTML, Image
import random
__counter__ = random.randint(0,2e9)

Image(url="best_bracket_2019.png")


In [ ]:
# df_tourneyCompact
df_tc = pd.read_csv(stage2DataPath + "NCAATourneyCompactResults.csv")
df_tc['ID'] = df_tc['Season'].astype(str) + '_' \
              + (np.minimum(df_tc['WTeamID'],df_tc['LTeamID'])).astype(str) + '_' \
              + (np.maximum(df_tc['WTeamID'],df_tc['LTeamID'])).astype(str)

df_tc['Result'] = 1*(df_tc['WTeamID'] < df_tc['LTeamID'])
# df_tourneyCompact.head(10)

df_tc = df_tc.merge(df_tourney_seeds.rename(columns={'Seed':'WSeed','TeamID':'WTeamID'}), 
                    how='inner', on=['Season', 'WTeamID'])
df_tc = df_tc.merge(df_tourney_seeds.rename(columns={'Seed':'LSeed','TeamID':'LTeamID'}), 
                    how='inner', on=['Season', 'LTeamID'])
# Remove any play-in games
# df_playin = df_tc[df_tc['WSeed'].str[0:3] == df_tc['LSeed'].str[0:3]]
df_tc.head(10)
# df_tc = df_tc[df_tc['WSeed'].str[0:3] != df_tc['LSeed'].str[0:3]]

In [ ]:
df_submission = pd.read_csv("best_model_results_pre2019.csv")
# df_submission.head()
df_score, df_analysis = score_submission(df_submission, df_tc, on_season = np.arange(2014,2019), return_df_analysis=True)
df_score

In [ ]:
df_score['LogLoss'].mean()